In [ ]:
import pandas as pd

In [ ]:
# read in our files

mvps = pd.read_csv(r"C:\Users\wrsno\Documents\NBA_MVPs\mvps.csv")
teams = pd.read_csv(r"C:\Users\wrsno\Documents\NBA_MVPs\teams\teams.csv")

In [ ]:
mvps


In [ ]:
del mvps["Unnamed: 0"]

In [ ]:
mvps.info()


In [ ]:
# Players that tied in voting have a 'T' next to their rank
# We need to remove that to do some sorting

mvps['Rank'] = mvps['Rank'].str.replace("T", '', regex = False)

In [ ]:
mvps[mvps['Rank'] == '1'].sort_values(by = 'PTS', ascending = True).head(33)

In [ ]:
# Need to remove * from team names in our team data frame
# the * denotes teams that made the playoffs

teams["Team"] = teams["Team"].str.replace("*", "", regex = False)

In [ ]:
# Because team names are stored as full names in the teams df and only abbreviations in the mvps df,
# we need to find a way to relate the two so we can join our tables

teams["Team"].unique()

In [ ]:
mvps["Tm"].unique()

In [ ]:
# we are going to use a csv file that contains both the abbreviation
# and the full name as a sort of key

nicknames = {}

with open(r"C:\Users\wrsno\Documents\NBA_MVPs\teams\nicknames.txt") as f:
    lines = f.readlines()
    for line in lines[1:]:                                      # skip the header row
        abbrev,name = line.replace("\n", "").split(",")         # get rid of "\n" and split on the comma
        nicknames[abbrev] = name                                # assign to the nickname dictionary

In [ ]:
nicknames

In [ ]:
# assign the dictionary values to the keys in our mvps table
# and name it a new column "Team" with the full team name
# now we can join the mvps df to the teams df on the Team column
mvps["Team"] = mvps["Tm"].map(nicknames)

In [ ]:
mvps

In [ ]:
# Check to see if there are any players whose Team is TOT, which indicates they played for 
# multiple teams that season, and won't have one of the Team abbreviations we just created

mvps[mvps['Tm'].str.contains('TOT')]

In [ ]:
# Merge our tables. Using a left join because there are a few players whose team is listed as TOT
# in the mvps table and we don't want to lose them
# if we did an inner join it would drop those values since there is no TOT team in the teams df

stats = mvps.merge(teams, how = "left", on=["Team", "Year"])

In [ ]:
# The number of rows in our new stats table should be the same as the mvps table
stats

In [ ]:
del stats["Unnamed: 0"]

In [ ]:
stats.dtypes

In [ ]:
# the dash in GB means a team is 0 games back, so replace '-' with 0
# so we can convert to a numeric data type
stats["GB"].unique()

In [ ]:
stats["GB"] = stats["GB"].str.replace("—","0")

In [ ]:
# Check Rank too, but I think we fixed this earlier when we dropped the 'T' from rankings
stats["Rank"].unique()

In [ ]:
stats["GB"] = pd.to_numeric(stats["GB"])
stats["Rank"] = pd.to_numeric(stats["Rank"])

In [ ]:
stats.info()

In [ ]:
stats.to_csv(r"C:\Users\wrsno\Documents\NBA_MVPs\stats.csv")

In [ ]:
# Exploring which stats are most correlated with highest share of first place MVP votes

stats.corr()["Share"]

In [ ]:
stats.corr()["Share"].plot.bar()